# SVMの実装

このノートブックでは、SVM（サポートベクターマシン）の実装と可視化を学習します。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)

# データの生成
X, y = make_classification(n_samples=100, n_features=2, n_redundant=0, n_informative=2, 
                         n_clusters_per_class=1, random_state=42)
X_scaled = StandardScaler().fit_transform(X)

# 異なるカーネルでのSVM
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, kernel in enumerate(kernels):
    # SVMの訓練
    svm = SVC(kernel=kernel, random_state=42)
    svm.fit(X_scaled, y)
    
    # 予測
    y_pred = svm.predict(X_scaled)
    accuracy = accuracy_score(y, y_pred)
    
    # 決定境界の可視化
    h = 0.02
    x_min, x_max = X_scaled[:, 0].min() - 1, X_scaled[:, 0].max() + 1
    y_min, y_max = X_scaled[:, 1].min() - 1, X_scaled[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    Z = svm.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    axes[i].contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.RdYlBu)
    axes[i].scatter(X_scaled[:, 0], X_scaled[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolors='black')
    axes[i].set_title(f'SVM ({kernel.capitalize()}) - Accuracy: {accuracy:.3f}')
    axes[i].set_xlabel('Feature 1')
    axes[i].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

# サポートベクトルの可視化
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_scaled, y)

plt.figure(figsize=(10, 8))
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y, cmap=plt.cm.RdYlBu, alpha=0.7)
plt.scatter(svm_rbf.support_vectors_[:, 0], svm_rbf.support_vectors_[:, 1], 
           s=100, facecolors='none', edgecolors='black', linewidth=2, label='Support Vectors')
plt.title('Support Vectors')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()

print("=== SVMの結果 ===")
print(f"サポートベクトルの数: {len(svm_rbf.support_vectors_)}")
print(f"サポートベクトルのインデックス: {svm_rbf.support_}")
print(f"各クラスのサポートベクトル数: {np.bincount(y[svm_rbf.support_])}")

# 異なるC値での性能
C_values = [0.1, 1, 10, 100]
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, C in enumerate(C_values):
    svm_c = SVC(kernel='rbf', C=C, random_state=42)
    svm_c.fit(X_scaled, y)
    y_pred = svm_c.predict(X_scaled)
    accuracy = accuracy_score(y, y_pred)
    
    # 決定境界の可視化
    h = 0.02
    x_min, x_max = X_scaled[:, 0].min() - 1, X_scaled[:, 0].max() + 1
    y_min, y_max = X_scaled[:, 1].min() - 1, X_scaled[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    Z = svm_c.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    axes[i].contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.RdYlBu)
    axes[i].scatter(X_scaled[:, 0], X_scaled[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolors='black')
    axes[i].set_title(f'SVM (C={C}) - Accuracy: {accuracy:.3f}')
    axes[i].set_xlabel('Feature 1')
    axes[i].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

print(f"\n=== 異なるC値での性能 ===")
for C in C_values:
    svm_c = SVC(kernel='rbf', C=C, random_state=42)
    svm_c.fit(X_scaled, y)
    y_pred = svm_c.predict(X_scaled)
    accuracy = accuracy_score(y, y_pred)
    print(f"C={C}: 精度={accuracy:.4f}, サポートベクトル数={len(svm_c.support_vectors_)}")
